In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df_glu_217_5p = pd.read_excel('~/Studio/tools/glu_24217_5p_6mG.xlsx')
df_glu_217_3p = pd.read_excel('~/Studio/tools/glu_24217_3p_6mG.xlsx')
df_glu_217_5p.shape, df_glu_217_3p.shape

In [ ]:
gln_path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx'
df_mock = load_data(gln_path)
gln_path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx'
df_rsv = load_data(gln_path)
df_mock.shape, df_rsv.shape

In [ ]:
gln_path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/SecondBatch/OE240_UTMB_Mock_5ul_9min_IPM_LP_420to2k_21Dec2020.xlsx'
df_mock2 = load_data(gln_path)
gln_path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/SecondBatch/OE240_UTMB_RSV_5ul_9min_IPM_LP_420to2k_21Dec2020.xlsx'
df_rsv2 = load_data(gln_path)
df_mock2_only, df_rsv2_only = diff_dfs(df_mock2, df_rsv2)
df_mock2.shape, df_rsv2.shape

In [ ]:
fpath = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/UTMB1.xlsx'
df_intact_mock = load_data(fpath)
fpath = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/UTMB6.xlsx'
df_intact_rsv = load_data(fpath)

df_intact_mock = df_intact_mock[(df_intact_mock.Mass>24000)&(df_intact_mock.Mass<24600)]
df_intact_rsv = df_intact_rsv[(df_intact_rsv.Mass>24000)&(df_intact_rsv.Mass<24600)]

In [ ]:
df_mock_24k = df_mock[(df_mock.Mass>24000)&(df_mock.Mass<24500)]
df_rsv_24k = df_rsv[(df_rsv.Mass>24000)&(df_rsv.Mass<24500)]
df_mock_24k_only, df_rsv_24k_only = diff_dfs(df_mock_24k, df_rsv_24k)

In [ ]:
_, df_only_intact_rsv = diff_dfs(df_intact_mock, df_intact_rsv)

In [ ]:
# plotly_zone(df_intact_mock, y='Vol')
# plotly_zone(df_intact_rsv, y='Vol')

# plotly_zone(df_mock_24k_only, y='Vol')

df_glu_217_5p
# plot_zones(df_mock, df_rsv, y='RT')

In [ ]:
df_mock = df_mock[df_mock.Mass>1500]
df_rsv = df_rsv[df_rsv.Mass>1500]

In [ ]:
# df_mock.describe(percentiles=[.75])
df_mock.Vol.quantile(.75)

In [ ]:
df_mock_sample = df_mock#[df_mock.Vol>df_mock.Vol.quantile(.75)].copy()
df_rsv_sample = df_rsv#[df_rsv.Vol>df_rsv.Vol.quantile(.75)].copy()
df_mock_sample.shape, df_rsv_sample.shape

In [ ]:
plotly_zone(df_rsv2, y='RT')
# plotly_zone(df_rsv, y='Vol')
# df_mock.shape, df_rsv.shape

In [ ]:
bcr = base_calling_random(df_rsv2, base_only=True)
plotly_basecalling(df_rsv2, bcr[1])

In [ ]:
dfm_comm_rsv = match_dfs(df_mock, df_rsv)
dfm_comm_mock = match_dfs(df_rsv, df_mock)
dfm_only_rsv = df_rsv.drop(dfm_comm_rsv.index)
dfm_only_mock = df_mock.drop(dfm_comm_mock.index)

In [ ]:
df_mock_top = local_top(df_mock)
df_rsv_top = local_top(df_rsv)
df_mock_top.shape, df_rsv_top.shape

In [ ]:
_, df_rsv_top_only = diff_dfs(df_mock_top, df_rsv_top)

In [ ]:
_, df_rsv_sample_only = diff_dfs(df_mock_sample, df_rsv_sample)
df_rsv_sample_only.shape

In [ ]:
# remove adducts from df_mock
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_mock, df_mock, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_mock_without_adducts = df_mock.drop(dfm_adducts.index)

In [ ]:
# remove adducts from df_rsv
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_rsv, df_rsv, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_rsv_without_adducts = df_rsv.drop(dfm_adducts.index)

In [ ]:
df_mock_sample = local_top(df_mock_without_adducts, top=3)
df_rsv_sample = local_top(df_rsv_without_adducts, top=3)
df_mock_sample.shape, df_rsv_sample.shape

In [ ]:
_, df_rsv_sample_only = diff_dfs(df_mock_sample, df_rsv_sample)
df_rsv_sample_only.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 16

# params = [(df_intact_rsv, df_intact_rsv, shift) for shift in np.arange(0, 300, 1)]
# params = [(df_glu_217_5p, df_rsv2, shift) for shift in np.arange(-300, 300, 1)]
# params = [(df_mock2, df_rsv2_only, shift) for shift in np.arange(-64, 64, 1)]
params = [(df_mock_sample, df_rsv_sample_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_mock, df_rsv, shift) for shift in np.arange(1, 300, 1)]
# params = [(df_mock_top, df_rsv_top_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_mock, dfm_only_rsv, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit<200], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
med_hit = df_shift_hits.hit.max() / 2
df_hm_top = df_shift_hits[df_shift_hits.hit > med_hit]
df_hm_top = df_hm_top.sort_values('hit', ascending=False)
df_hm_top

In [ ]:
def func_count_basecallings(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

def func_longest_ladder(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    groups = basecalling_groups(df)
    return len(groups[0][1]) if groups else 0

df_s1 = df_wild_sample.copy()
df_s2 = df_alkb_sample.copy()
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[0:10].copy()
df_sh_top['basecallings'] = df_sh_top.apply(
    lambda x: func_count_basecallings(x['shift'], df_s1, df_s2), axis=1)
df_sh_top['longestladder'] = df_sh_top.apply(
    lambda x: func_longest_ladder(x['shift'], df_s1, df_s2), axis=1)
df_sh_top

In [ ]:
def func_count_basecallings(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

def func_longest_ladder(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    groups = basecalling_groups(df)
    return len(groups[0][1]) if groups else 0

df_s1 = df_mock_sample.copy()
df_s2 = df_rsv_sample.copy()
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[0:10].copy()
df_sh_top['basecallings'] = df_sh_top.apply(
    lambda x: func_count_basecallings(x['shift'], df_s1, df_s2), axis=1)
df_sh_top['longestladder'] = df_sh_top.apply(
    lambda x: func_longest_ladder(x['shift'], df_s1, df_s2), axis=1)
df_sh_top

In [ ]:
shift = -14
dfm_base_l, dfm_base_r = peer_dfs(df_mock_sample, df_rsv_sample, shift=0)
# dfm_l, dfm_r = peer_dfs(df_mock_sample, df_rsv_sample, shift=shift)
# dfm_l, dfm_r = peer_dfs(df_mock_sample, df_mock_sample, shift=shift)
dfm_l, dfm_r = peer_dfs(df_rsv_sample, df_rsv_sample, shift=shift)
# dfm_l.shape, dfm_r.shape
plotly_zones(dfm_base_l, dfm_l)
bcr = base_calling_random(dfm_l)
plotly_basecalling(dfm_l, bcr[1], y='RT')
bcr = base_calling_random(dfm_r)
plotly_basecalling(dfm_r, bcr[1], y='RT')

In [ ]:
# df_sh_mock2rsv = df_shift_hits.copy()
df_sh_intact_mock2rsv = df_shift_hits.copy()

In [ ]:
df_tmp_mock = df_mock[(df_mock.Mass>7360-3*M)&(df_mock.Mass<7362+M)]
df_tmp_rsv = df_rsv[(df_rsv.Mass>7360-4*M)&(df_rsv.Mass<7362+M)]
plotly_zone(df_tmp_mock, y='Vol')
plotly_zone(df_tmp_rsv, y='Vol')

In [ ]:
shift = -28

_, dfm_l = peer_dfs(df_mock_sample, df_mock_sample, shift=shift)
# _, dfm_l = peer_dfs(df_rsv_sample, df_rsv_sample, shift=shift)
_, dfm_r = peer_dfs(df_mock_sample, df_rsv_sample, shift=shift)
dfm_l.shape, dfm_r.shape

In [ ]:
shift = -1

# dfm_r = match_dfs(df_mock, dfm_only_rsv, shift=shift)
# dfm_l = match_dfs(dfm_only_rsv, df_mock, shift=-1*shift)

dfm_mock = match_dfs(df_intact_mock, df_only_intact_rsv, shift=shift)
dfm_rsv = match_dfs(df_rsv, df_rsv, shift=shift)

# dfm.shape, dfm_mock.shape, dfm_rsv.shape

In [ ]:
shifts = df_hm_top['shift'].tolist()
dfm_list = list()
for shift in shifts:
    dfm_ = match_dfs(df_mock_sample, df_rsv_sample_only, shift=shift)
    dfm_list.append(dfm_)
# dfm_list.append(df_mock)
dfm_shifts = pd.concat(dfm_list).drop_duplicates()

In [ ]:
# bcr = base_calling_random(dfm_shifts, base_only=True)
# plotly_basecalling(dfm_shifts, bcr[1])
# plotly_zone(dfm_shifts)

dft = dfm_list[0]
plotly_zone(dfm_l)
plotly_zone(dfm_r)
# plotly_zones(dfm_l, dfm_r, y='RT')
# plotly_zones(dfm_r, dft)
dfm_r.shape


In [ ]:
dfm_l.shape

In [ ]:
def func(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

df_s1 = df_mock_sample.copy()
df_s2 = df_rsv_sample_only.copy()
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[:10].copy()
df_sh_top['basecallings'] = df_sh_top.apply(lambda x: func(x['shift'], df_s1, df_s2), axis=1)
df_sh_top

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))
df_plt = df_sh_top.copy().sort_values('shift')
df_plt1 = df_plt.copy()
df_plt1['shift'] = df_plt1['shift'].astype(int)
df_plt1 = df_plt1.set_index('shift')
df_plt1.plot(kind='bar',figsize=(2.5, 1.5))
plt.legend()

fig.tight_layout()
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
plt.savefig(svg_fpath, transparent=True, dpi=300)